# Youtube Video Assistant

---

## 1. Importing the required libraries

In [22]:
import os
import textwrap

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

## 2. Setting Up the API Key

In [41]:
# setting up the API token
API_KEY = "AI-----------------------------------PJ"
os.environ["API_KEY"] = API_KEY

In [24]:
API_KEY = os.environ.get("API_Key")

## 3. Building the YouTube Video Assistant

In [25]:
def create_db_from_video_url(video_url):
    """
    Creates an Embedding of the Video and performs 
    """
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)

    loader = YoutubeLoader.from_youtube_url(video_url)
    transcripts = loader.load()
    # cannot provide this directly to the model so we are splitting the transcripts into small chunks

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcripts)

    db = FAISS.from_documents(docs, embedding=embeddings)

    return db

In [26]:
db = create_db_from_video_url("https://www.youtube.com/watch?v=C82lT9cWQiA")
db

In [27]:
def get_response(api_key, db, request, k=5):
    """
    Usind Gemini Pro to get the response. It can handle upto 32k.
    """

    docs = db.similarity_search(query=request, k=k)
    docs_content = " ".join([doc.page_content for doc in docs])

    chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, convert_system_message_to_human=True)

    # creating a template for request
    template = """
    You are an assistant that can answer questions about youtube videos based on
    video transcripts: {docs}

    Only use factual information from the transcript to answer the question.

    If you don't have enough information to answer the question, say "I don't know".

    Your Answers should be detailed.
    """

    system_msg_prompt = SystemMessagePromptTemplate.from_template(template)

    # human prompt
    human_template = "Answer the following questions: {question}"
    human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_msg_prompt, human_msg_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=request, docs=docs_content)

    return response, docs

In [28]:
video_url = "https://www.youtube.com/watch?v=Q8d4FGWZDbE"
db = create_db_from_video_url(video_url=video_url)

request = "What is he talking about? and mention each phones specialities"
response, docs = get_response(API_KEY, db, request)

In [29]:
# using textwrap for printing neatly
print(textwrap.fill(response, width=100))

1. What is he talking about?  He is comparing the design, camera system, performance, and audio of
the iPhone and the Xiaomi.   2. What are the iPhone's specialities?  - Simple and elegant design -
Consistent and accurate colors in photos - Direct and loud vocals in audio - Familiar and intuitive
software that works with muscle memory developed on the iPhone   3. What are the Xiaomi's
specialities?  - Unique and edgy design - High-quality camera system with super high-quality lenses,
first-class focusing, and pleasing colors - Better graphics and stability under extreme stress test
- Symmetrical sound with one bottom-firing and one top-firing speaker - New MIUI 14 that allows for
opening more apps at once but still may be too similar to Apple's software for some users - Unibody
design with reduced points of vulnerability and a specially treated silicone rug on top that is
resistant to smudges, fingerprints, and yellowing


In [30]:
# video transcripts
docs

[Document(page_content="the design of these phones the iPhone is much less polarizing it kind of has to be if Apple wants this to be the phone that 30 million people are going to buy they've got no room to be edgy with any of the style choices they've got to keep it simple clean elegant and this does that xiaomi doesn't have that same concern I think this company knows full well that they won't be selling 30 million units of these they're going to sell two maybe three million and those three million are going to be Pro users who are actively seeking something that feels unique and this achieves that goal I wouldn't say it's the neatest prettiest phone it's got like a speed bump halfway up the back of it besides the phone blend into the rear which is again just a bit weird and it kind of makes this strip going up the back look like a carpet that's not fitted properly the phone rattles quite a bit thanks to lots of large camera lenses moving around and the screen borders are still not co

## 4. Creating a Gradio Web App

In [8]:
import gradio, langchain, langchain_google_genai, faiss

In [20]:
gradio.__version__, langchain.__version__, faiss.__version__

('4.8.0', '0.0.345', '1.7.4')

In [21]:
%%writefile requirements.txt
gradio==4.8.0
langchain==0.0.345
faiss-cpu==1.7.4
langchain_google_genai
youtube_transcript_api

Overwriting requirements.txt


In [32]:
%%writefile app.py
import gradio as gr
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

def create_db_from_video_url(video_url, api_key):
    """
    Creates an Embedding of the Video and performs 
    """
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

    loader = YoutubeLoader.from_youtube_url(video_url)
    transcripts = loader.load()
    # cannot provide this directly to the model so we are splitting the transcripts into small chunks

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcripts)

    db = FAISS.from_documents(docs, embedding=embeddings)
    return db

def get_response(video, request):
    """
    Usind Gemini Pro to get the response. It can handle upto 32k tokens.
    """
    API_KEY = os.environ.get("API_Key")
    db = create_db_from_video_url(video, API_KEY)
    docs = db.similarity_search(query=request, k=5)
    docs_content = " ".join([doc.page_content for doc in docs])

    chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=API_KEY, convert_system_message_to_human=True)

    # creating a template for request
    template = """
    You are an assistant that can answer questions about youtube videos based on
    video transcripts: {docs}

    Only use factual information from the transcript to answer the question.

    If you don't have enough information to answer the question, say "I don't know".

    Your Answers should be detailed.
    """

    system_msg_prompt = SystemMessagePromptTemplate.from_template(template)

    # human prompt
    human_template = "Answer the following questions: {question}"
    human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_msg_prompt, human_msg_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=request, docs=docs_content)

    return response

# creating title, description for the web app
title = "YouTube Video Assistant 🧑‍💻"
description = "Answers to the Questions asked by the user on the specified YouTube video. (English Only).\n\n"\
"Click here to view [demo](https://cdn-uploads.huggingface.co/production/uploads/641aa7814577db917f70f8aa/A-AN-Nkt4qGUKvPhJ92dJ.mp4)."
article = "Other Projects:<br/>"\
"💰 [Health Insurance Predictor](http://health-insurance-cost-predictor-k19.streamlit.app/)<br/>"\
"📰 [Fake News Detector](https://fake-news-detector-k19.streamlit.app/)<br/>"\
"🪶 [Birds Classifier](https://huggingface.co/spaces/Kathir0011/Birds_Classification)"

# building the app
youtube_video_assistant = gr.Interface(
    fn=get_response,
    inputs=[gr.Text(label="Enter the Youtube Video URL:", placeholder="Example: https://www.youtube.com/watch?v=MnDudvCyWpc"),
            gr.Text(label="Enter your Question", placeholder="Example: What's the video is about?")],
    outputs=gr.TextArea(label="Answers using Gemini Pro:"),
    title=title,
    description=description,
    article=article
)

# launching the web app
youtube_video_assistant.launch()


Overwriting app.py


In [1]:
# embedding the deployed app to notebook
from IPython.display import IFrame

IFrame(src="https://kathir0011-youtube-video-assistant.hf.space", width=800, height=600)